# Beginning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import darts
from darts import TimeSeries
from darts.models import NaiveSeasonal, NaiveMean, NaiveDrift
from darts.models import StatsForecastAutoARIMA, StatsForecastAutoETS, StatsForecastAutoCES, RNNModel, ExponentialSmoothing, BlockRNNModel
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from tqdm import tqdm_notebook as tqdm
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape, mase, mse, rmse, ase, ape, r2_score, smape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dropout, Dense, Flatten
import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)
import random
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Nadam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
from tensorflow.keras.losses import Huber
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import tensorflow as tf
from tensorflow.keras import backend as K
import os
K.clear_session()
tf.keras.backend.clear_session()
from hyperopt import base
os.environ['PYTHONHASHSEED'] = '0'
os.environ['HYPEROPT_FMIN_SEED'] = "1"
random.seed(88)
np.random.seed(88)
tf.random.set_seed(88)
base.have_been_bugged = False  
rstate = np.random.default_rng(88)

/Users/jacobzhao/anaconda3/envs/prophet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
2024-06-15 17:13:35.602834: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df_total = pd.read_csv('time_series_thesis_question_1.csv', index_col=0)
df_total.index = pd.to_datetime(df_total.index)

In [3]:
df = TimeSeries.from_dataframe(df_total)

In [4]:
train_1, val_1 = df.split_before(pd.Timestamp('20230101'))
train_2, val_2 = df.split_before(pd.Timestamp('20230401'))
train_3, val_3 = df.split_before(pd.Timestamp('20230701'))

In [5]:
def Xy(sliding_windows):
    X = [[list(window[:-1]) for window in windows] for windows in sliding_windows]
    y = [[window[-1] for window in windows] for windows in sliding_windows]
    return np.array(X), np.array(y)

In [6]:
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error

def spliter(df_total,
            k = 4,
            test_size = 3,
            val_size = 3):
    test = []
    val = []
    train = []
    NN_sets = {}
    window_size = k+1


    for col in df_total.columns:
        windows = [np.array(window) for window in df_total[col].rolling(window_size) if len(window) == window_size]
        test.append(windows[-(test_size):])
        val.append(windows[-(test_size+val_size):-(test_size)])
        train.append(windows[:-(test_size+val_size)])
    NN_sets['X_train'], NN_sets['y_train'] = Xy(train)
    NN_sets['X_val'], NN_sets['y_val'] = Xy(val)
    NN_sets['X_test'], NN_sets['y_test'] = Xy(test)
    return NN_sets


In [7]:
import darts
def NN_metricker(y_pred):
  y_pred_df = pd.DataFrame(y_pred.reshape((-1, 3)).transpose())
  y_pred_df.columns = df_total.columns
  y_pred_df.index = df_total.index[-3:]
  y_pred_tf = TimeSeries.from_dataframe(y_pred_df)
  SMAPE = darts.metrics.smape(val_1, y_pred_tf)
  MASE = darts.metrics.mase(val_1, y_pred_tf, train_1)
  MAE = darts.metrics.mae(val_1, y_pred_tf)
  print(
      "Symmetric Mean absolute percentage error: {:.2f}%.".format(
          SMAPE),
          "MASE: {:.2f}".format(MASE),
          "MAE: {:.2f}".format(MAE)
      )
  return y_pred_df, SMAPE, MASE, MAE


# RQ1 Results from CNN

In [11]:
def create_2cnn_model(window, n_layers, n_nodes, optimizer_name, lr, kernel_size, df=df_total):
    NN_sets = spliter(df, k=window)

    model = Sequential()
    model.add(Conv1D(filters=n_nodes, kernel_size= kernel_size, activation='relu', input_shape=(window, 1)))
    model.add(Conv1D(filters=n_nodes/2, kernel_size=kernel_size, activation='relu', input_shape=(window, 1)))
    model.add(Flatten())
    
    for _ in range(n_layers):
        model.add(Dense(n_nodes/2, activation='relu'))
        
    model.add(Dense(1))

    optimizer_class = {'adam': Adam, 'rmsprop': RMSprop, 'sgd': SGD, 'nadam': Nadam}[optimizer_name]
    optimizer = optimizer_class(lr)

    model.compile(optimizer=optimizer, loss=Huber())

    return model, NN_sets

In [29]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Nadam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
K.clear_session()
tf.keras.backend.clear_session()

# Hyperparameters to tune
window_values = [16, 20, 24, 28]
hidden_layers_values = [1, 2, 3]
hidden_nodes_values = [8, 16, 32, 64]
optimizer_values = ['adam', 'rmsprop', 'nadam']
kernel_size_values = [3, 5, 7, 9, 12]
lr_values = list(np.arange(1e-4, 11e-4, 1e-4))

trials_results = []
def is_compatible(window, kernel_size):
    if window == 16 and kernel_size >= 9:
        return False
    elif window == 20 and kernel_size >= 12:
        return False
    return True

def objective(params):
    window = int(params['window'])
    n_layers = int(params['n_layers'])
    n_nodes = int(params['n_nodes'])
    optimizer_name = params['optimizer']
    lr = params['lr']
    kernel_size = int(params['kernel_size'])

    if not is_compatible(window, kernel_size):
        return {'status': STATUS_OK, 'loss': np.inf}


    model, NN_sets = create_2cnn_model(window, n_layers, n_nodes, optimizer_name, lr, kernel_size)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(
        NN_sets['X_train'].reshape((-1, NN_sets['X_train'].shape[-1])),
        NN_sets['y_train'].reshape((-1, 1)),
        epochs=30,
        batch_size=16,
        validation_data=(NN_sets['X_val'].reshape((-1, NN_sets['X_val'].shape[-1])),
                         NN_sets['y_val'].reshape((-1, 1))),
        callbacks=[early_stopping],
        verbose=0
    )
    val_loss = history.history['val_loss'][-1]
    trials_results.append({'params': params, 'val_loss': val_loss})

    return {'loss': val_loss, 'status': STATUS_OK}

# Define the hyperparameter search space
search_space = {
    'window': hp.choice('window', window_values),
    'n_layers': hp.choice('n_layers', hidden_layers_values),
    'n_nodes': hp.choice('n_nodes', hidden_nodes_values),
    'optimizer': hp.choice('optimizer', optimizer_values),
    'lr': hp.choice('lr', lr_values),
    'kernel_size': hp.choice('kernel_size', kernel_size_values)  
}
# Conduct the Bayesian optimization
trials = Trials()
best = fmin(objective, search_space, algo=tpe.suggest, max_evals=100, trials=trials, rstate=rstate)

# Print the best hyperparameters and the corresponding validation loss
print('Best hyperparameters:')
print(best)



  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 100/100 [24:59<00:00, 14.99s/trial, best loss: 0.8905212879180908]
Best hyperparameters:
{'kernel_size': 0, 'lr': 4, 'n_layers': 1, 'n_nodes': 1, 'optimizer': 2, 'window': 1}


In [30]:
K.clear_session()
tf.keras.backend.clear_session()
# Map the indices back to the actual values
best_window = window_values[best['window']]
best_n_layers = hidden_layers_values[best['n_layers']]
best_n_nodes = hidden_nodes_values[best['n_nodes']]
best_optimizer_name = optimizer_values[best['optimizer']]
best_lr = lr_values[best['lr']]
best_kernel_size = kernel_size_values[best['kernel_size']]

print(f"Best Window: {best_window}")
print(f"Best Number of Layers: {best_n_layers}")
print(f"Best Number of Nodes: {best_n_nodes}")
print(f"Best Optimizer Name: {best_optimizer_name}")
print(f"Best Learning Rate: {best_lr}")
print(f"Best Kernel Size: {best_kernel_size}")
best_model, NN_sets = create_2cnn_model(best_window, best_n_layers, best_n_nodes, best_optimizer_name, best_lr, best_kernel_size)
early_stop = EarlyStopping(monitor='val_loss', patience=5)

history = best_model.fit(NN_sets['X_train'].reshape((-1, best_window)),
                         NN_sets['y_train'].reshape((-1, 1)),
                         epochs=30,
                         batch_size=16,
                         validation_data=(NN_sets['X_val'].reshape((-1, best_window)),
                                          NN_sets['y_val'].reshape((-1, 1))),
                         callbacks=[early_stop])

Best Window: 20
Best Number of Layers: 2
Best Number of Nodes: 16
Best Optimizer Name: nadam
Best Learning Rate: 0.0005
Best Kernel Size: 3
Epoch 1/30
628/628 [==============================] - 4s 3ms/step - loss: 1.8693 - val_loss: 1.3749
Epoch 2/30
628/628 [==============================] - 1s 2ms/step - loss: 0.9318 - val_loss: 1.1227
Epoch 3/30
628/628 [==============================] - 1s 2ms/step - loss: 0.7911 - val_loss: 1.0306
Epoch 4/30
628/628 [==============================] - 2s 2ms/step - loss: 0.7446 - val_loss: 0.9630
Epoch 5/30
628/628 [==============================] - 2s 2ms/step - loss: 0.7255 - val_loss: 0.9275
Epoch 6/30
628/628 [==============================] - 2s 3ms/step - loss: 0.7199 - val_loss: 0.9523
Epoch 7/30
628/628 [==============================] - 1s 2ms/step - loss: 0.7177 - val_loss: 0.9108
Epoch 8/30
628/628 [==============================] - 1s 2ms/step - loss: 0.7153 - val_loss: 0.9096
Epoch 9/30
628/628 [==============================] - 2s 3ms

In [31]:
y_pred = best_model.predict(NN_sets['X_test'].reshape((-1, best_window, 1)))
y_pred_df = NN_metricker(y_pred)

73/73 [==============================] - 0s 2ms/step
Symmetric Mean absolute percentage error: 14.73%. MASE: 1.12 MAE: 1.16


In [32]:
import pandas as pd

# Assuming trials_results is your list of dictionaries
df = pd.DataFrame(trials_results)

# If you want to flatten the 'params' column into separate columns
df = pd.concat([df.drop(['params'], axis=1), df['params'].apply(pd.Series)], axis=1)
df.head()

,val_loss,kernel_size,lr,n_layers,n_nodes,optimizer,window
0,0.953209,3,0.0010,2,16,adam,28
1,1.013165,3,0.0003,3,16,rmsprop,20
2,0.925028,7,0.0009,2,8,rmsprop,24
3,0.910332,3,0.0001,2,32,adam,20
4,1.044035,7,0.0009,3,64,adam,28


In [33]:
import os

# Create directory if it doesn't exist
if not os.path.exists('final_thesis_CNN'):
    os.makedirs('final_thesis_CNN')

# Write df to a csv file in the specified directory
df.to_csv('final_thesis_CNN/CNN_tuning_results.csv', index=False)

In [9]:
best = {'kernel_size': 0, 'lr': 4, 'n_layers': 1, 'n_nodes': 1, 'optimizer': 2, 'window': 1}
# Hyperparameters to tune
window_values = [16, 20, 24, 28]
hidden_layers_values = [1, 2, 3]
hidden_nodes_values = [8, 16, 32, 64]
optimizer_values = ['adam', 'rmsprop', 'nadam']
kernel_size_values = [3, 5, 7, 9, 12]
lr_values = list(np.arange(1e-4, 11e-4, 1e-4))

best_window = window_values[best['window']]
best_n_layers = hidden_layers_values[best['n_layers']]
best_n_nodes = hidden_nodes_values[best['n_nodes']]
best_optimizer_name = optimizer_values[best['optimizer']]
best_lr = lr_values[best['lr']]
best_kernel_size = kernel_size_values[best['kernel_size']]

In [12]:
SMAPE_values = []
MASE_values = []
MAE_values = []

for i in range(20):
    K.clear_session()
    tf.keras.backend.clear_session()
    best_model, NN_sets = create_2cnn_model(best_window, best_n_layers, best_n_nodes, best_optimizer_name, best_lr, best_kernel_size)
    early_stop = EarlyStopping(monitor='val_loss', patience=5)
    history = best_model.fit(
        NN_sets['X_train'].reshape((-1, best_window)),
        NN_sets['y_train'].reshape((-1, 1)),
        epochs=30,
        batch_size=16,
        validation_data=(NN_sets['X_val'].reshape((-1, best_window)),
                         NN_sets['y_val'].reshape((-1, 1))),
        callbacks=[early_stop]
    )
    y_pred = best_model.predict(NN_sets['X_test'].reshape((-1, best_window)))
    y_pred_df, SMAPE, MASE, MAE = NN_metricker(y_pred)
    
    # Append the metrics to the lists
    SMAPE_values.append(SMAPE)
    MASE_values.append(MASE)
    MAE_values.append(MAE)

628/628 [==============================] - 2s 3ms/step - loss: 0.7128 - val_loss: 0.8939
Epoch 9/30
628/628 [==============================] - 2s 3ms/step - loss: 0.7121 - val_loss: 0.9501
Epoch 10/30
628/628 [==============================] - 2s 3ms/step - loss: 0.7115 - val_loss: 0.8986
Epoch 11/30
628/628 [==============================] - 2s 3ms/step - loss: 0.7091 - val_loss: 0.9148
Epoch 12/30
628/628 [==============================] - 2s 4ms/step - loss: 0.7097 - val_loss: 0.9128
Epoch 13/30
628/628 [==============================] - 2s 3ms/step - loss: 0.7087 - val_loss: 0.8900
Epoch 14/30
628/628 [==============================] - 2s 3ms/step - loss: 0.7072 - val_loss: 0.8879
Epoch 15/30
628/628 [==============================] - 2s 3ms/step - loss: 0.7062 - val_loss: 0.8867
Epoch 16/30
628/628 [==============================] - 2s 3ms/step - loss: 0.7043 - val_loss: 0.8927
Epoch 17/30
628/628 [==============================] - 2s 3ms/step - loss: 0.7055 - val_loss: 0.8974
Epo

In [13]:
import os
import pickle

# Create directory if it doesn't exist
if not os.path.exists('final_thesis_CNN'):
    os.makedirs('final_thesis_CNN')

# Save the lists to pickle files in the specified directory
with open('final_thesis_CNN/CNN_SMAPE_values.pkl', 'wb') as f:
    pickle.dump(SMAPE_values, f)
with open('final_thesis_CNN/CNN_MASE_values.pkl', 'wb') as f:
    pickle.dump(MASE_values, f)
with open('final_thesis_CNN/CNN_MAE_values.pkl', 'wb') as f:
    pickle.dump(MAE_values, f)